Устанавливаем нужные зависимости с нужными версиями

In [1]:
# !apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2
# !pip uninstall -y -q tensorflow keras tensorflow-estimator tensorflow-text
# !pip install -q tensorflow_datasets
# !pip install -q -U tensorflow-text tensorflow

In [1]:
# Импортируем нужные модули
import logging
import time
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
import tensorflow_datasets as tfds
import tensorflow_text as text
import tqdm

import pickle
import warnings
warnings.filterwarnings('ignore')

2023-03-17 13:53:22.481117: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Загружаем данные взятые из соревнование Text Normalization Challenge - Russian Language
# Ссылка на датасет https://www.kaggle.com/competitions/text-normalization-challenge-russian-language/data
# Я перевел их в нужный формат;

# Тренировочные данные
with open('train_list', 'rb') as f:
    data_list = pickle.load(f)
    
x_sentence_list = data_list[0]
y_sentence_list = data_list[1]

# Тестовые данные которые предоставил преподаватель
with open('test/TextTrain.txt', 'r') as f:
    test = f.readlines()
    
with open('test/TextTrue.txt', 'r') as f:
    true = f.readlines()

clean_test = list(map(lambda x: x.replace('\n', '').encode('utf-8'), test))
clean_true = list(map(lambda x: x.replace('\n', '').encode('utf-8'), true))

# Конвертируем наши листы с предложениями в tf dataset
train_tensor = tf.data.Dataset.from_tensor_slices((x_sentence_list, y_sentence_list))
val_tensor = tf.data.Dataset.from_tensor_slices((clean_test, clean_true))


2023-03-17 13:53:32.676203: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
for before, after in train_tensor.batch(3).take(1):
    print('\t Примеры до нормализации:')
    for bf in before.numpy():
        print(bf.decode('utf-8'))
    print()
    print('\t Примеры после нормализации:')
    for aft in after.numpy():
        print(aft.decode('utf-8'))

	 Примеры до нормализации:
По состоянию на 1862 год .
Оснащались латными рукавицами и сабатонами с не длинными носками .
В конце 1811 года , вследствие конфликта с проезжим вельможей ( графом Салтыковым ) вынужден был оставить службу по личному прошению .

	 Примеры после нормализации:
По состоянию на тысяча восемьсот шестьдесят второй год .
Оснащались латными рукавицами и сабатонами с не длинными носками .
В конце тысяча восемьсот одиннадцатого года , вследствие конфликта с проезжим вельможей ( графом Салтыковым ) вынужден был оставить службу по личному прошению .


In [4]:
for before, after in val_tensor.batch(3).take(1):
    print('\t Примеры до нормализации:')
    for bf in before.numpy():
        print(bf.decode('utf-8'))
    print()
    print('\t Примеры после нормализации:')
    for aft in after.numpy():
        print(aft.decode('utf-8'))

	 Примеры до нормализации:
Было опрошено 187 человек, в т.ч. 91 женщина, 67 мужчин и 29 подростков.
В 1660 г. король Людовик XIV женился на инфанте Марии-Терезии Австрийской. 
Рек длиной более 10 км. насчитывается в области 348, их суммарная длина составляет 10235 км.

	 Примеры после нормализации:
Было опрошено сто восемьдесят семь человек, в том числе девяносто одна женщина, шестьдесят семь мужчин и двадцать девять подростков.
В тысяча шестьсот шестидесятый год король Людовик четырнадцатый женился на инфанте Марии-Терезии Австрийской. 
Рек длиной более десяти километров насчитывается в области триста сорок восемь, их суммарная длина составляет десять тысяч двести тридцать пять километров


In [5]:
# Define a tokenizer class
import pathlib
import re

bert_tokenizer_params = dict(lower_case=True)
reserved_tokens = ["[PAD]", "[UNK]", "[START]", "[END]"]

START = tf.argmax(tf.constant(reserved_tokens) == "[START]")
END = tf.argmax(tf.constant(reserved_tokens) == "[END]")

def cleanup_text(reserved_tokens, token_txt):
    # Remove reserved tokens except for "[UNK]".
    bad_tokens = [re.escape(tok) for tok in reserved_tokens if tok != "[UNK]"]
    bad_token_re = "|".join(bad_tokens)

    bad_cells = tf.strings.regex_full_match(token_txt, bad_token_re)
    result = tf.ragged.boolean_mask(token_txt, ~bad_cells)

    # Join the tokens into strings.
    result = tf.strings.reduce_join(result, separator=' ', axis=-1)

    return result

def add_start_end(ragged):
    count = ragged.bounding_shape()[0]
    starts = tf.fill([count,1], START)
    ends = tf.fill([count,1], END)
    return tf.concat([starts, ragged, ends], axis=1)

class CustomTokenizer(tf.Module):
    def __init__(self, reserved_tokens, vocab_path):
        self.tokenizer = text.BertTokenizer(vocab_path, lower_case=True)
        self._reserved_tokens = reserved_tokens
        self._vocab_path = tf.saved_model.Asset(vocab_path)

        vocab = pathlib.Path(vocab_path).read_text().splitlines()
        self.vocab = tf.Variable(vocab)

        ## Create signatures for export:   

        # Include a tokenize signature for a batch of strings. 
        self.tokenize.get_concrete_function(
            tf.TensorSpec(shape=[None], dtype=tf.string))

        # Include `detokenize` and `lookup` signatures for:
        #   * `Tensors` with shapes [tokens] and [batch, tokens]
        #   * `RaggedTensors` with shape [batch, tokens]
        self.detokenize.get_concrete_function(
            tf.TensorSpec(shape=[None, None], dtype=tf.int64))
        self.detokenize.get_concrete_function(
            tf.RaggedTensorSpec(shape=[None, None], dtype=tf.int64))

        self.lookup.get_concrete_function(
            tf.TensorSpec(shape=[None, None], dtype=tf.int64))
        self.lookup.get_concrete_function(
            tf.RaggedTensorSpec(shape=[None, None], dtype=tf.int64))

        # These `get_*` methods take no arguments
        self.get_vocab_size.get_concrete_function()
        self.get_vocab_path.get_concrete_function()
        self.get_reserved_tokens.get_concrete_function()

    @tf.function
    def tokenize(self, strings):
        enc = self.tokenizer.tokenize(strings)
        # Merge the `word` and `word-piece` axes.
        enc = enc.merge_dims(-2, -1)
        enc = add_start_end(enc)
        return enc

    @tf.function
    def detokenize(self, tokenized):
        words = self.tokenizer.detokenize(tokenized)
        return cleanup_text(self._reserved_tokens, words)

    @tf.function
    def lookup(self, token_ids):
        return tf.gather(self.vocab, token_ids)

    @tf.function
    def get_vocab_size(self):
        return tf.shape(self.vocab)[0]

    @tf.function
    def get_vocab_path(self):
        return self._vocab_path

    @tf.function
    def get_reserved_tokens(self):
        return tf.constant(self._reserved_tokens)


In [6]:
# Создаем токенизатор на основе слов на русском собранных из TED Talks
tokenizers = tf.Module()
tokenizers.ru = CustomTokenizer(reserved_tokens, 'ru_vocab.txt')

model_name = 'ted_hrlr_translate_ru_en_converter'
tf.saved_model.save(tokenizers, model_name);

INFO:tensorflow:Assets written to: ted_hrlr_translate_ru_en_converter/assets


### Создадим пайплайн данных

In [7]:
# Define a function to prepare a batch of inputs and labels
def preprocess_batch(source, target):
    # Tokenize the source text
    source_tokens = tokenizers.ru.tokenize(source)
    source_tokens = source_tokens[:, :128]
    source_tensor = source_tokens.to_tensor()

    # Tokenize the target text
    target_tokens = tokenizers.ru.tokenize(target)
    target_tokens = target_tokens[:, :(128 + 1)]

    # Prepare the inputs and labels for the decoder
    target_inputs = target_tokens[:, :-1].to_tensor()
    target_labels = target_tokens[:, 1:].to_tensor()

    return (source_tensor, target_inputs), target_labels

def make_batches(ds):
    return (
        ds.shuffle(20000)
          .batch(64)
          .map(preprocess_batch, tf.data.AUTOTUNE)
          .prefetch(buffer_size=tf.data.AUTOTUNE))

# Create training and validation set batches.
train_batches = make_batches(train_tensor)
val_batches = make_batches(val_tensor)

In [8]:
def positional_encoding(length, depth):
    depth = depth/2

    positions = np.arange(length)[:, np.newaxis]     
    depths = np.arange(depth)[np.newaxis, :]/depth
  
    angle_rates = 1 / (10000**depths)
    angle_rads = positions * angle_rates

    pos_encoding = np.concatenate(
      [np.sin(angle_rads), np.cos(angle_rads)],
      axis=-1) 

    return tf.cast(pos_encoding, dtype=tf.float32)

class PositionalEmbedding(tf.keras.layers.Layer):
    def __init__(self, vocab_size, d_model):
        super().__init__()
        self.d_model = d_model
        self.embedding = tf.keras.layers.Embedding(vocab_size, d_model, mask_zero=True) 
        self.pos_encoding = positional_encoding(length=2048, depth=d_model)

    def compute_mask(self, *args, **kwargs):
        return self.embedding.compute_mask(*args, **kwargs)

    def call(self, x):
        length = tf.shape(x)[1]
        x = self.embedding(x)
        # This factor sets the relative scale of the embedding and positonal_encoding.
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x = x + self.pos_encoding[tf.newaxis, :length, :]
        return x

class BaseAttention(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__()
        self.mha = tf.keras.layers.MultiHeadAttention(**kwargs)
        self.layernorm = tf.keras.layers.LayerNormalization()
        self.add = tf.keras.layers.Add()

class CrossAttention(BaseAttention):
    def call(self, x, context):
        attn_output, attn_scores = self.mha(
            query=x,
            key=context,
            value=context,
            return_attention_scores=True)
   
        # Cache the attention scores for plotting later.
        self.last_attn_scores = attn_scores

        x = self.add([x, attn_output])
        x = self.layernorm(x)
    
        return x

class GlobalSelfAttention(BaseAttention):
    def call(self, x):
        attn_output = self.mha(
            query=x,
            value=x,
            key=x)
        x = self.add([x, attn_output])
        x = self.layernorm(x)
        return x

class CausalSelfAttention(BaseAttention):
    def call(self, x):
        attn_output = self.mha(
            query=x,
            value=x,
            key=x,
            use_causal_mask = True)
        x = self.add([x, attn_output])
        x = self.layernorm(x)
        return x

    
class FeedForward(tf.keras.layers.Layer):
    def __init__(self, d_model, dff, dropout_rate=0.1):
        super().__init__()
        self.seq = tf.keras.Sequential([
          tf.keras.layers.Dense(dff, activation='relu'),
          tf.keras.layers.Dense(d_model),
          tf.keras.layers.Dropout(dropout_rate)
        ])
        self.add = tf.keras.layers.Add()
        self.layer_norm = tf.keras.layers.LayerNormalization()

    def call(self, x):
        x = self.add([x, self.seq(x)])
        x = self.layer_norm(x) 
        return x


In [9]:
class Encoder(tf.keras.layers.Layer):
    def __init__(self, *, num_layers, d_model, num_heads, dff, vocab_size, dropout_rate=0.1):
        super().__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        self.pos_embedding = PositionalEmbedding(
            vocab_size=vocab_size, d_model=d_model)

        self.self_attention_layers = [
            GlobalSelfAttention(num_heads=num_heads, key_dim=d_model, dropout=dropout_rate)
            for _ in range(num_layers)]

        self.ffn_layers = [
            FeedForward(d_model, dff)
            for _ in range(num_layers)]

        self.dropout = tf.keras.layers.Dropout(dropout_rate)

    def call(self, x):
        x = self.pos_embedding(x)

        x = self.dropout(x)

        for i in range(self.num_layers):
            x = self.self_attention_layers[i](x)
            x = self.ffn_layers[i](x)

        return x

class Decoder(tf.keras.layers.Layer):
    def __init__(self, *, num_layers, d_model, num_heads, dff, vocab_size,
               dropout_rate=0.1):
        super(Decoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        self.pos_embedding = PositionalEmbedding(vocab_size=vocab_size,
                                                 d_model=d_model)
        self.dropout = tf.keras.layers.Dropout(dropout_rate)

        self.causal_self_attention_layers = [
            CausalSelfAttention(num_heads=num_heads, key_dim=d_model, dropout=dropout_rate)
            for _ in range(num_layers)]

        self.cross_attention_layers = [
            CrossAttention(num_heads=num_heads, key_dim=d_model, dropout=dropout_rate)
            for _ in range(num_layers)]

        self.ffn_layers = [
            FeedForward(d_model, dff)
            for _ in range(num_layers)]

        self.last_attn_scores = None

    def call(self, x, context):
    
        x = self.pos_embedding(x)

        x = self.dropout(x)

        for i in range(self.num_layers):
            x = self.causal_self_attention_layers[i](x)
            x = self.cross_attention_layers[i](x, context)
            self.last_attn_scores = self.cross_attention_layers[i].last_attn_scores
            x = self.ffn_layers[i](x)

        return x

class Transformer(tf.keras.Model):
    def __init__(self, *, num_layers, d_model, num_heads, dff,
               input_vocab_size, target_vocab_size, dropout_rate=0.1):
        super().__init__()
        self.encoder = Encoder(num_layers=num_layers, d_model=d_model,
                               num_heads=num_heads, dff=dff,
                               vocab_size=input_vocab_size,
                               dropout_rate=dropout_rate)

        self.decoder = Decoder(num_layers=num_layers, d_model=d_model,
                               num_heads=num_heads, dff=dff,
                               vocab_size=target_vocab_size,
                               dropout_rate=dropout_rate)

        self.final_layer = tf.keras.layers.Dense(target_vocab_size)

    def call(self, inputs):
        context, x  = inputs
        context = self.encoder(context) 
        x = self.decoder(x, context)  
        logits = self.final_layer(x)
        
        try:
            del logits._keras_mask
        except AttributeError:
            pass

        return logits


In [11]:
# Зададим гиперпараметры
num_layers = 4
d_model = 128
dff = 512
num_heads = 8
dropout_rate = 0.1

transformer = Transformer(
    num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    dff=dff,
    input_vocab_size=tokenizers.ru.get_vocab_size().numpy(),
    target_vocab_size=tokenizers.ru.get_vocab_size().numpy(),
    dropout_rate=dropout_rate)

In [12]:
for (bef, aft), aft_labels in train_batches.take(1):
    break

output = transformer((bef, aft))

print(bef.shape)
print(aft.shape)
print(output.shape)

(64, 86)
(64, 128)
(64, 128, 7832)


In [15]:
# выведем структуру сети
transformer.summary()

Model: "transformer"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder (Encoder)           multiple                  3641344   
                                                                 
 decoder (Decoder)           multiple                  5752320   
                                                                 
 dense_16 (Dense)            multiple                  1010328   
                                                                 
Total params: 10,403,992
Trainable params: 10,403,992
Non-trainable params: 0
_________________________________________________________________


## Тренировка сети

In [21]:
# Define a loss function that computes the masked sparse categorical cross-entropy loss
def masked_loss(labels, predictions):
    mask = labels != 0
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')
    losses = loss_fn(labels, predictions)
    mask = tf.cast(mask, dtype=losses.dtype)
    losses *= mask
    loss = tf.reduce_sum(losses) / tf.reduce_sum(mask)
    return loss

# Define a function that computes the masked accuracy
def masked_accuracy(labels, predictions):
    predicted_labels = tf.argmax(predictions, axis=2)
    labels = tf.cast(labels, predicted_labels.dtype)
    correct_matches = labels == predicted_labels
    mask = labels != 0
    correct_matches = correct_matches & mask
    correct_matches = tf.cast(correct_matches, dtype=tf.float32)
    mask = tf.cast(mask, dtype=tf.float32)
    accuracy = tf.reduce_sum(correct_matches) / tf.reduce_sum(mask)
    return accuracy

# Define a custom learning rate schedule for the Adam optimizer
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super().__init__()
        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)
        self.warmup_steps = warmup_steps

    def __call__(self, step):
        step = tf.cast(step, dtype=tf.float32)
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)
        lr = tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)
        return lr
    
# Create an instance of the CustomSchedule class and use it to create an Adam optimizer
learning_rate_schedule = CustomSchedule(d_model)
optimizer = tf.keras.optimizers.Adam(learning_rate_schedule, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

transformer.compile(
    loss=masked_loss,
    optimizer=optimizer,
    metrics=[masked_accuracy])

transformer.fit(train_batches,
                epochs=20,
                validation_data=val_batches)

In [29]:
class Translator(tf.Module):
    """Class for getting network predictions"""
    def __init__(self, tokenizers, transformer):
        self.tokenizers = tokenizers
        self.transformer = transformer

    def __call__(self, sentence, max_length=128):
        assert isinstance(sentence, tf.Tensor)
        if len(sentence.shape) == 0:
            sentence = sentence[tf.newaxis]

        sentence = self.tokenizers.pt.tokenize(sentence).to_tensor()

        encoder_input = sentence

        start_end = self.tokenizers.en.tokenize([''])[0]
        start = start_end[0][tf.newaxis]
        end = start_end[1][tf.newaxis]

        output_array = tf.TensorArray(dtype=tf.int64, size=0, dynamic_size=True)
        output_array = output_array.write(0, start)

        for i in range(max_length):
            output = tf.transpose(output_array.stack())
            predictions = self.transformer([encoder_input, output], training=False)

            predictions = predictions[:, -1:, :]

            predicted_id = tf.argmax(predictions, axis=-1)

            output_array = output_array.write(i+1, predicted_id[0])

            if predicted_id == end:
                break

        output = tf.transpose(output_array.stack())
        text = self.tokenizers.ru.detokenize(output)[0]

        tokens = self.tokenizers.ru.lookup(output)[0]
        self.transformer([encoder_input, output[:,:-1]], training=False)
        attention_weights = self.transformer.decoder.last_attn_scores

        return text, tokens, attention_weights

    
def calculate_bleu_scores(translator, sentences, references):
    """
    Calculates BLEU score for each sentence translated by the given translator 
    against its corresponding reference sentence. Returns the average BLEU score.
    """
    from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
    
    clean_references = [r.replace('\n', '').split(' ') for r in references]
    preds = []
    sum_bleu = 0
    
    for sentence in sentences:
        translated_text, translated_tokens, attention_weights = translator(tf.constant(sentence))
        preds.append(translated_text)
        
    for i in range(len(clean_references)):
        bleu = sentence_bleu(clean_references[i], preds[i], smoothing_function=SmoothingFunction(epsilon=1).method7)
        sum_bleu += bleu
    
    return sum_bleu / len(clean_references)


with open('test/TextTrue.txt', 'r') as f:
    true = f.readlines()

with open('TextTrain.txt', 'r') as f:
    train = f.readlines()

with open('TextTest.txt', 'r') as f:
    test = f.readlines()

clean_train = [t.replace('\n', '') for t in train]
clean_test = [t.replace('\n', '') for t in test]

translator = Translator(tokenizers, transformer)

bleu_train = calculate_bleu_scores(translator, clean_train, true)
bleu_test = calculate_bleu_scores(translator, clean_test, true)

print('BLEU for stt_normalizer: ', bleu_test)
print('BLEU for non normalized text: ', bleu_train)


## Экспорт модели

In [31]:
class ExportTranslator(tf.Module):
    def __init__(self, translator):
        self.translator = translator

    @tf.function(input_signature=[tf.TensorSpec(shape=[], dtype=tf.string)])
    def __call__(self, sentence):
        (result,
         tokens,
         attention_weights) = self.translator(sentence, max_length=MAX_TOKENS)

        return result

translator = ExportTranslator(translator)
tf.saved_model.save(translator, export_dir='translator')

In [32]:
reloaded = tf.saved_model.load('translator')
reloaded('185 г. до Христа').numpy()